In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import fingerprint_enhancer
import random
import albumentations as A


In [ ]:
data_dir = "./Public_Dataset"
img_size = (300, 400)
batch_size = 16
num_classes = 4

In [ ]:
# Display dataset information
def display_dataset_info(data_dir):
    class_names = os.listdir(data_dir)  # List all class directories
    class_count = {class_name: len(os.listdir(os.path.join(data_dir, class_name))) for class_name in class_names}
    
    # Print the number of images in each class
    print("Dataset Class Distribution:")
    for class_name, count in class_count.items():
        print(f"{class_name}: {count} images")
    
    # Show a sample image from each class
    plt.figure(figsize=(12, 8))
    for idx, class_name in enumerate(class_names):
        class_path = os.path.join(data_dir, class_name)
        sample_image_path = os.path.join(class_path, os.listdir(class_path)[0])  # Get the first image from the class
        
        # Read and resize the image
        img = cv2.imread(sample_image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img = cv2.resize(img, img_size)
        
        # Display the image
        plt.subplot(2, 4, idx + 1)
        plt.imshow(img)
        plt.title(class_name)
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Call the function to display dataset information
display_dataset_info(data_dir)

In [ ]:
import cv2
import numpy as np
import random

def simple_augment(image):
    # --- Rotation ---
    angle = random.uniform(-10, 10)
    h, w = image.shape
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
    image = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REFLECT)

    # --- Brightness ---
    brightness_factor = random.uniform(0.8, 1.2)
    image = np.clip(image * brightness_factor, 0, 255)

    # --- Contrast ---
    contrast_factor = random.uniform(0.8, 1.2)
    mean = np.mean(image)
    image = np.clip((image - mean) * contrast_factor + mean, 0, 255)

    return image.astype(np.uint8)


In [ ]:
def apply_masked_processing(image):
    """
    Apply Gaussian blur and adaptive thresholding to the fingerprint region.
    More conservative approach to preserve ridge patterns.
    """
    # Step 1: Create a binary mask for the fingerprint region
    # Use a more conservative threshold to preserve more detail
    _, binary_mask = cv2.threshold(image, 180, 255, cv2.THRESH_BINARY_INV)
    
    # Step 2: Apply lighter Gaussian blur to preserve ridge details
    blurred_image = cv2.GaussianBlur(image, (3, 3), 0.5)  # Reduced sigma
    
    # Step 3: Apply adaptive thresholding with adjusted parameters
    threshold_image = cv2.adaptiveThreshold(
        blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 15, 8  # Increased block size and C value
    )
    
    # Step 4: Isolate the fingerprint area (where mask is white)
    enhanced_fingerprint = cv2.bitwise_and(threshold_image, threshold_image, mask=binary_mask)
    
    # Step 5: Keep the original background (where mask is black)
    background = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(binary_mask))
    
    # Step 6: Combine the enhanced fingerprint and the original background
    final_image = cv2.add(enhanced_fingerprint, background)
    
    # Ensure the result has sufficient variation
    if np.std(final_image) < 5:
        print("Warning: Masked processing resulted in low variation, returning original")
        return image
    
    return final_image

In [ ]:

def preprocess_image(image_path, img_size, augment=False):
    """
    Loads, resizes, preprocesses, optionally augments, and enhances the fingerprint image.
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    assert image is not None, f"Could not read image: {image_path}"
    image = cv2.resize(image, img_size)

    # Apply masked processing
    image = apply_masked_processing(image)

    # Apply manual augmentation
    if augment:
        image = simple_augment(image)

    # Check if image has sufficient variation before enhancement
    if np.std(image) < 10:  # Very low variation
        print(f"Warning: Low variation in image {image_path}, skipping enhancement")
        # Skip enhancement for problematic images
        pass
    else:
        try:
            # Fingerprint enhancement with error handling
            enhanced = fingerprint_enhancer.enhance_fingerprint(image)
            # Only use enhanced version if it's valid
            if enhanced is not None and np.std(enhanced) > 0:
                image = enhanced
            else:
                print(f"Warning: Enhancement failed for {image_path}, using original")
        except Exception as e:
            print(f"Warning: Enhancement error for {image_path}: {e}")
            # Continue with original processed image

    # Normalize and expand dims
    image = (image * 255).astype(np.uint8) if image.dtype != np.uint8 else image
    image = image / 255.0
    image = np.expand_dims(image, axis=-1)

    return image

In [ ]:
class CustomImageGenerator(Sequence):
    def __init__(self, data_dir, img_size=(300, 400), batch_size=16, subset='training', validation_split=0.2, shuffle=True):
        self.data_dir = data_dir
        self.img_size = img_size
        self.batch_size = batch_size
        self.subset = subset
        self.validation_split = validation_split
        self.shuffle = shuffle
        self.class_names = sorted(os.listdir(data_dir))
        self.class_indices = {class_name: idx for idx, class_name in enumerate(self.class_names)}
        self.filepaths, self.labels = self._load_dataset()
        self.on_epoch_end()
    
    def _load_dataset(self):
        filepaths = []
        labels = []
        for class_name in self.class_names:
            class_path = os.path.join(self.data_dir, class_name)
            image_files = os.listdir(class_path)
            image_paths = [os.path.join(class_path, f) for f in image_files]
            n_total = len(image_paths)
            n_val = int(n_total * self.validation_split)
            if self.subset == 'training':
                selected = image_paths[n_val:]
            else:
                selected = image_paths[:n_val]
            filepaths.extend(selected)
            labels.extend([self.class_indices[class_name]] * len(selected))
        return filepaths, labels
    
    def __len__(self):
        return len(self.filepaths) // self.batch_size

    
    def __getitem__(self, index):
        batch_paths = self.filepaths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]
        
        images = [preprocess_image(path, self.img_size) for path in batch_paths]
        labels_one_hot = to_categorical(batch_labels, num_classes=len(self.class_names))
        
        return np.array(images), np.array(labels_one_hot)
    
    def on_epoch_end(self):
        if self.shuffle:
            combined = list(zip(self.filepaths, self.labels))
            random.shuffle(combined)
            self.filepaths, self.labels = zip(*combined)


In [ ]:
# Instantiate training and validation generators
train_gen = CustomImageGenerator(data_dir=data_dir, img_size=img_size, batch_size=batch_size, subset='training', validation_split=0.2)
val_gen = CustomImageGenerator(data_dir=data_dir, img_size=img_size, batch_size=batch_size, subset='validation', validation_split=0.2)


In [ ]:
from collections import Counter

train_counts = Counter(train_gen.labels)
valid_counts = Counter(val_gen.labels)

print("Training samples per class:")
for class_idx, count in train_counts.items():
    class_name = train_gen.class_names[class_idx]
    print(f"  {class_name}: {count}")

print("Validation samples per class:")
for class_idx, count in valid_counts.items():
    class_name = val_gen.class_names[class_idx]
    print(f"  {class_name}: {count}")


In [ ]:
%matplotlib


In [ ]:
def visualize_samples(generator, num_samples=4):
    """
    Visualize original, processed, and enhanced images from the generator.
    """
    sample_paths = random.sample(generator.filepaths, num_samples)
    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4 * num_samples))
    
    for i, img_path in enumerate(sample_paths):
        # Load original image
        img_orig = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img_orig = cv2.resize(img_orig, generator.img_size)
        
        # Apply masked processing
        img_processed = apply_masked_processing(img_orig)
        
        # Enhance fingerprint
        img_enhanced = fingerprint_enhancer.enhance_fingerprint(img_processed)
        img_enhanced = (img_enhanced * 255).astype(np.uint8)
        
        # Plot images
        axes[i, 0].imshow(img_orig, cmap='gray')
        axes[i, 0].set_title('Original')
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(img_processed, cmap='gray')
        axes[i, 1].set_title('Masked + Gaussian')
        axes[i, 1].axis('off')
        
        axes[i, 2].imshow(img_enhanced, cmap='gray')
        axes[i, 2].set_title('Enhanced')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize samples from the training generator
visualize_samples(train_gen, num_samples=4)

In [ ]:
# Check the class indices mapping
print("Class Indices Mapping:", train_gen.class_indices)

In [ ]:
print("Class Indices Mapping:", val_gen.class_indices)


In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Use train_gen.labels instead of train_gen.classes
class_labels = np.array(train_gen.labels)

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(class_labels),
    y=class_labels
)

# Optional: Map weights to class indices
class_weights_dict = dict(enumerate(class_weights))

print("Class weights:", class_weights_dict)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Build a Convolutional Neural Network (CNN) Model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(img_size[0], img_size[1], 1)),
    BatchNormalization(),  # Normalize activations after convolution
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation="relu"),
    BatchNormalization(),  # Normalizing dense layer activations
    Dropout(0.5),  # Regularization to prevent overfitting

    Dense(num_classes, activation="softmax")  # Output layer
])


In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return K.mean(K.sum(loss, axis=-1))
    return focal_loss_fixed

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=["accuracy"]
)


In [ ]:
# Print the model summary
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
# Callback to reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Metric to monitor
    factor=0.5,          # Factor by which the learning rate will be reduced
    patience=3,          # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6          # Lower bound on the learning rate
)

In [ ]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

In [ ]:
steps_per_epoch = len(train_gen)
validation_steps = len(val_gen)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[reduce_lr, early_stopping],
    class_weight=class_weights_dict,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    verbose=1
)


In [ ]:
# Example for a Keras Sequence or generator
batch = next(iter(train_gen))
print("Batch type:", type(batch))
print("Batch length:", len(batch))
print("X batch shape:", batch[0].shape)
print("y batch shape:", batch[1].shape)


In [ ]:
batch_val = next(iter(val_gen))
print("Validation batch X shape:", batch_val[0].shape)
print("Validation batch y shape:", batch_val[1].shape)
